# csv transformare

In [79]:
import fitz  # PyMuPDF
import io
import pytesseract
from PIL import Image
import pdfplumber
import csv
import pandas as pd
from collections import defaultdict


def extract_table_titles(pdf_path):
    # Deschide PDF-ul
    doc = fitz.open(pdf_path)
    table_titles = []
    title_frequencies = defaultdict(int)

    # Variabilă pentru a ține evidența rândurilor goale între titlurile de tabele
    blank_lines_count = 0
    
    # Parcurge fiecare pagină
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = text.split('\n')
        
        for line in lines:
            words = line.split()
            
            # Verifică dacă linia este goală
            if len(words) == 0:
                blank_lines_count += 1
            else:
                # Verifică dacă linia conține un singur cuvânt care începe cu literă mare
                if len(words) == 1 and words[0][0].isupper():
                    # Pentru primul titlu de tabel, nu este nevoie să verificăm numărul de rânduri goale
                    if not table_titles or blank_lines_count >= 2:
                        table_titles.append(words[0])
                        title_frequencies[words[0]] = 0
                    blank_lines_count = 0  # Resetează contorul de rânduri goale
                else:
                    # Resetează contorul de rânduri goale dacă întâlnește o linie care nu este goală sau nu este titlu de tabel
                    blank_lines_count = 0

    return table_titles, dict(title_frequencies)
    
def extract_images_from_pdf(pdf_path, output_folder):
    pdf_document = fitz.open(pdf_path)
    index=0
    
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        image_list = page.get_images(full=True)

        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]

            image = Image.open(io.BytesIO(image_bytes))
            image_path = os.path.join(output_folder,f"Image{index + 1}.jpg")
            index=index+1
            image.save(image_path)

            print(f"Saved image: {image_path}")
            

    print("Image extraction complete.")


def normalize_header(header):
    """Normalizează header-ul eliminând spațiile și caracterele de nouă linie."""
   # header=header.rstrip('\n')
    # header.replace('\n',"").strip()
    for df in header.columns:
        df=df.replace('\n','').strip()
    return header#header.replace('\n','').strip()

def extract_tables_from_pdf(pdf_path, output_folder):
    table_titles, title_frequencies = extract_table_titles(pdf_path)
    index=0
    all_tables_df = pd.DataFrame()
    # Cuvinte cheie de verificat în antetul tabelului (normalizate)
    keywords = {"region_id", "country_id", "location_id", "job_id"}
    
    # Creează directorul de ieșire dacă nu există
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            tables = page.extract_tables()

            for table_index, table in enumerate(tables):
                # Creează DataFrame din tabel
                if len(table) > 1:
                    df = pd.DataFrame(table[1:], columns=table[0])
                else:
                    continue  # Sari peste tabelele fără date
                
                # Normalizează antetul
                headers = [header for header in df.columns] #{normalize_header(header) for header in df.columns} # [header for header in df.columns] 
                #df=normalize_header(df)
                #headers2=[header for header in df.columns] 
                print(f"Page {page_num + 1}, Table {table_index + 1} headers: {headers}")
                
                # Verifică dacă antetul conține toate cuvintele cheie
                ok = 0
                for keyword in keywords:
                    if keyword in headers:
                        ok = 1
                        break

                if ok == 1:
                    # Construiește calea fișierului folosind os.path.join
                    table_path = os.path.join(output_folder, f"{table_titles[index]}.csv")
                    index=index+1
                    aux_header=headers
                    #df2=df
                    df.to_csv(table_path, index=False)
                    if all_tables_df.empty:
                            all_tables_df = df
                    else:
                            all_tables_df = pd.concat([all_tables_df, df], ignore_index=True)
                    
                    
                else: 
                    previous_table_path = os.path.join(output_folder, f"{table_titles[index - 1]}.csv")
                    df_existent = pd.read_csv(previous_table_path)
                    df.loc[-1] = df.columns  # Adaugă antetul inițial ca prima linie
                    df.index = df.index + 1  # Mută toate indexurile în jos
                    df = df.sort_index()  
                    df.columns = aux_header
                    df_existent = pd.concat([df_existent, df], ignore_index=True)
                    #print(df_existent)
                    df_existent.to_csv(previous_table_path, index=False)
                    
    

                
                print(f"Saved table: {table_path}, OK: {ok}")
   
    print("Table extraction complete.")
    
# Example usage
pdf_path = 'Employee-details-1.pdf'  # Path to your PDF file
output_folder = 'extracted_content'  # Output folder to save images and tables
output_folder2='extracted_images'
import os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(output_folder2):
    os.makedirs(output_folder2)

# ok
extract_images_from_pdf(pdf_path, output_folder2)
extract_tables_from_pdf(pdf_path, output_folder)
import pandas as pd

# Load the provided CSV file
path = r"C:/Users/Talent2/Desktop/ness/extracted_content"
dir_list = os.listdir(path)
for file in dir_list:
    if file.endswith(".csv"):
        df_existent = pd.read_csv(os.path.join(path, file))
        print(df_existent.head())


#df_existent = pd.read_csv(csv_path)

# Display the first few rows of the dataframe
#print(df_existent.head())



Saved image: extracted_images\Image1.jpg
Image extraction complete.
Page 1, Table 1 headers: ['region_id', 'region_name']
Saved table: extracted_content\Regions.csv, OK: 1
Page 1, Table 2 headers: ['country_id', 'country_name', 'region_id']
Saved table: extracted_content\Countries.csv, OK: 1
Page 2, Table 1 headers: ['location_id', 'street_address', 'postal_code', 'city', 'state_province', 'country_id']
Saved table: extracted_content\Locations.csv, OK: 1
Page 3, Table 1 headers: ['department_id', 'department_name', 'manager_id', 'location_id']
Saved table: extracted_content\Departments.csv, OK: 1
Page 4, Table 1 headers: ['job_id', 'job_title', 'min_salary', 'max_salary']
Saved table: extracted_content\Jobs.csv, OK: 1
Page 5, Table 1 headers: ['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary', 'commission_pct', 'manager_id', 'department_id']
Saved table: extracted_content\Employees.csv, OK: 1
Page 6, Table 1 headers: ['132', 'TJ', 'Olson

# in caz ca vrei sa iti afisezi csv 

In [36]:
import pandas as pd

# Load the provided CSV file
csv_path = r"C:/Users/Talent2/Desktop/ness/extracted_content/Employees.csv"
df_existent = pd.read_csv(csv_path)

# Display the first few rows of the dataframe
print(df_existent.head())


   employee_id first_name last_name     email  phone_number    hire_date  \
0        100.0     Steven      King     SKING  515.123.4567  17-JUN-1987   
1        101.0      Neena   Kochhar  NKOCHHAR  515.123.4568  21-SEP-1989   
2        102.0        Lex   De Haan   LDEHAAN  515.123.4569  13-JAN-1993   
3        103.0  Alexander    Hunold   AHUNOLD  590.423.4567  03-JAN-1990   
4        104.0      Bruce     Ernst    BERNST  590.423.4568  21-MAY-1991   

    job_id   salary  commission_pct  manager_id  ...  Charles  Johnson  \
0  AD_PRES  24000.0             NaN         NaN  ...      NaN      NaN   
1    AD_VP  17000.0             NaN       100.0  ...      NaN      NaN   
2    AD_VP  17000.0             NaN       100.0  ...      NaN      NaN   
3  IT_PROG   9000.0             NaN       102.0  ...      NaN      NaN   
4  IT_PROG   6000.0             NaN       103.0  ...      NaN      NaN   

  CJOHNSON 011.44.1644.429262 04-JAN-2000 SA_REP 6200 .10  149  80  
0      NaN                NaN

# pentru procesare de imagini
## aici trebuie concatenat intr un string ce contine 'e' si dupa se pot face embeddings 

In [76]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Setarea căii către executabilul Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Modificați calea după locația instalării Tesseract

def preprocess_image(image_path):
    # Citește imaginea
    image = cv2.imread(image_path)
    
    # Verifică dacă imaginea a fost încărcată corect
    if image is None:
        raise FileNotFoundError(f"Imaginea nu a putut fi găsită la calea specificată: {image_path}")
    
    # Convertirea imaginii în nuanțe de gri
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Inversarea imaginii
    inverted = cv2.bitwise_not(gray)
    
    # Aplicarea unui filtru de umbrire pentru a îmbunătăți contrastul
    _, thresholded = cv2.threshold(inverted, 150, 255, cv2.THRESH_BINARY)
    
    # Salvarea imaginii preprocesate pentru verificare (opțional)
    preprocessed_path = 'preprocessed_image.png'
    cv2.imwrite(preprocessed_path, thresholded)
    print(f"Imagine preprocesată salvată la: {preprocessed_path}")
    
    return thresholded

def extract_text_from_image(image_path):
    try:
        # Preprocesarea imaginii
        preprocessed_image = preprocess_image(image_path)
        
        # Convertirea imaginii preprocesate la un format compatibil cu PIL
        pil_image = Image.fromarray(preprocessed_image)
        
        # Utilizarea Tesseract pentru a extrage textul
        text = pytesseract.image_to_string(pil_image)
        return text
    except Exception as e:
        print(f"Eroare la extragerea textului: {e}")
        return ""

# Exemplu de utilizare
image_path = r"C:\Users\Talent2\Desktop\ness\extracted_images\Image1.jpg"
try:
    extracted_text = extract_text_from_image(image_path)
    print("Extracted Text:", extracted_text)
except FileNotFoundError as e:
    print(e)


Imagine preprocesată salvată la: preprocessed_image.png
Extracted Text: John Doe

Executive Director
phone number 148.284.3886




# scurta verificare pentru chatbot

In [66]:
import cv2
import pytesseract
from PIL import Image
import numpy as np
import os
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Modificați calea după locația instalării Tesseract

def preprocess_image(image_path):
    # Citește imaginea
    image = cv2.imread(image_path)
    
    # Verifică dacă imaginea a fost încărcată corect
    if image is None:
        raise FileNotFoundError(f"Imaginea nu a putut fi găsită la calea specificată: {image_path}")
    
    # Convertirea imaginii în nuanțe de gri
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Inversarea imaginii
    inverted = cv2.bitwise_not(gray)
    
    # Aplicarea unui filtru de umbrire pentru a îmbunătăți contrastul
    _, thresholded = cv2.threshold(inverted, 150, 255, cv2.THRESH_BINARY)
    
    # Salvarea imaginii preprocesate pentru verificare (opțional)
    preprocessed_path = 'preprocessed_image.png'
    cv2.imwrite(preprocessed_path, thresholded)
    print(f"Imagine preprocesată salvată la: {preprocessed_path}")
    
    return thresholded

def extract_text_from_image(image_path):
    try:
        # Preprocesarea imaginii
        preprocessed_image = preprocess_image(image_path)
        
        # Convertirea imaginii preprocesate la un format compatibil cu PIL
        pil_image = Image.fromarray(preprocessed_image)
        
        # Utilizarea Tesseract pentru a extrage textul
        text = pytesseract.image_to_string(pil_image)
        return text
    except Exception as e:
        print(f"Eroare la extragerea textului: {e}")
        return ""

# Exemplu de utilizare
image_path = r"C:\Users\Talent2\Desktop\ness\extracted_images\Image1.jpg"
try:
    extracted_text = extract_text_from_image(image_path)
    #print("Extracted Text:", extracted_text)
except FileNotFoundError as e:
    print(e)

documents = [Document(page_content=extracted_text, metadata={"source": "Image1.jpg"})]

# Load CSV files using LangChain's CSVLoader
path = r"extracted_content"


# Create OpenAIEmbeddings instance
api_key = os.getenv('OPENAI_API_KEY')  # Replace with your OpenAI API key
embedding = OpenAIEmbeddings(api_key=api_key)

# ChromaDB setup
persist_directory = 'chroma_db'
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb.persist()  # Persist the vector database to disk

# RAG setup
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

# Example query
query = "What is John Doe phone number?"
answer = qa.invoke(query)
print("Answer:", answer)


Imagine preprocesată salvată la: preprocessed_image.png
Answer: {'query': 'What is John Doe phone number?', 'result': "John Doe's phone number is 148.284.3886."}


# build relations

In [70]:
import fitz  # PyMuPDF
import io
import pytesseract
from PIL import Image
import pdfplumber
import csv
import pandas as pd
from collections import defaultdict
import os
from langchain.schema import Document
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import DataFrameLoader

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Modificați calea după locația instalării Tesseract

def preprocess_image(image_path):
    # Citește imaginea
    image = cv2.imread(image_path)
    
    # Verifică dacă imaginea a fost încărcată corect
    if image is None:
        raise FileNotFoundError(f"Imaginea nu a putut fi găsită la calea specificată: {image_path}")
    
    # Convertirea imaginii în nuanțe de gri
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Inversarea imaginii
    inverted = cv2.bitwise_not(gray)
    
    # Aplicarea unui filtru de umbrire pentru a îmbunătăți contrastul
    _, thresholded = cv2.threshold(inverted, 150, 255, cv2.THRESH_BINARY)
    
    # Salvarea imaginii preprocesate pentru verificare (opțional)
    preprocessed_path = 'preprocessed_image.png'
    cv2.imwrite(preprocessed_path, thresholded)
    print(f"Imagine preprocesată salvată la: {preprocessed_path}")
    
    return thresholded

def extract_text_from_image(image_path):
    try:
        # Preprocesarea imaginii
        preprocessed_image = preprocess_image(image_path)
        
        # Convertirea imaginii preprocesate la un format compatibil cu PIL
        pil_image = Image.fromarray(preprocessed_image)
        
        # Utilizarea Tesseract pentru a extrage textul
        text = pytesseract.image_to_string(pil_image)
        return text
    except Exception as e:
        print(f"Eroare la extragerea textului: {e}")
        return ""

# Exemplu de utilizare
image_path = r"extracted_images\Image1.jpg"
try:
    extracted_text = extract_text_from_image(image_path)
    extracted_t="These are the only informations about John Doe: " + extracted_text
    #print("Extracted Text:", extracted_text)
except FileNotFoundError as e:
    print(e)

documents = [Document(page_content=extracted_t, metadata={"source": "Image1.jpg"})]

path = r"C:/Users/Talent2/Desktop/ness/extracted_content"
countries_df = pd.read_csv(os.path.join(path, 'Countries.csv'))
departments_df = pd.read_csv(os.path.join(path, 'Departments.csv'))
employees_df = pd.read_csv(os.path.join(path, 'Employees.csv'))
jobs_df = pd.read_csv(os.path.join(path, 'Jobs.csv'))
locations_df = pd.read_csv(os.path.join(path, 'Locations.csv'))
regions_df = pd.read_csv(os.path.join(path, 'Regions.csv'))
p1 = r"extracted_content/Countries.csv"
p2 = r"extracted_content/Departments.csv"
p3 = r"extracted_content/Employees.csv"
p4 = r"extracted_content/Jobs.csv"
p5 = r"extracted_content/Locations.csv"
p6 = r"extracted_content/Regions.csv"
merged_df = pd.merge(regions_df, countries_df, on='region_id')
merged_df = pd.merge(merged_df, locations_df, on='country_id')
merged_df = pd.merge(merged_df, departments_df, on='location_id')
merged_df = pd.merge(merged_df, employees_df, on='department_id')
merged_df = pd.merge(merged_df, employees_df, on='job_id')


# Print columns in merged_df to verify 'text' column existence
#print("Columns in merged_df:", merged_df.columns)

# Access 'text' column
merged_df['text'] = merged_df.astype(str).apply(' '.join, axis=1)
text_column = merged_df['text']

loader = DataFrameLoader(data_frame=merged_df, page_content_column='text')
documents = documents + loader.load()

# Create OpenAIEmbeddings instance
api_key = os.getenv('OPENAI_API_KEY')  # Replace with your OpenAI API key
embedding = OpenAIEmbeddings(api_key=api_key)

# ChromaDB setup
persist_directory = 'chroma_db'
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb.persist()  # Persist the vector database to disk

# RAG setup
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

# Example query
query = "What is John Doe phone number?"
try:
    answer = qa.invoke(query)
    print("Answer:", answer)
except Exception as e:
    print(f"Error occurred during query execution: {e}")

print("ok")

Imagine preprocesată salvată la: preprocessed_image.png
Answer: {'query': 'What is John Doe phone number?', 'result': "John Doe's phone number is 515.124.4269."}
ok
